In [4]:
import geopandas as gpd
from osgeo import gdal, ogr
import os

In [15]:



# Input GeoPandas object file path
#input_shp = 'C:/Geotar/CHAD/geodata/Raw/Roads/tcd_trs_roads_ocha/tcd_trs_roads_ocha.shp'
# input_shp = f"zip://C:/Geotar/{pilot}/geodata/Raw/Roads/irq_roads.zip/IRQ_roads.shp"
#mask_shp = "C:/Geotar/CHAD/geodata/Processed/Mask/Chad_mask.shp"
#f'C:/Geotar/{pilot}/geodata/Processed/Mask/Najaf_mask.shp'
pilot = "COL"

#input_shp = f'C:/Geotar/{pilot}/geodata/Raw/Roads/Red_nal_dep.shp'
input_shp = f'C:/Geotar/{pilot}/geodata/Raw/health/Colombia-node.shp'
mask_shp = f'C:/Geotar/{pilot}/geodata/Processed/Mask/COL_mask.shp'

# Output raster file path
#output_file = f"C:/Geotar/{pilot}/geodata/workspace/rast_road.tif"
output_file = f"C:/Geotar/{pilot}/geodata/workspace/rast_health.tif"

# Read the GeoPandas object
gdf = gpd.read_file(input_shp)
mask = gpd.read_file(mask_shp)
# Clip the func_gdf using the mask
gdf = gpd.clip(gdf, mask)

gdf.to_file(f"C:/Geotar/{pilot}/geodata/workspace/health_clip.shp")

C:\Users\oscar.bautista\AppData\Local\Temp\ipykernel_9320\2266012215.py:22: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(f"C:/Geotar/{pilot}/geodata/workspace/health_clip.shp")


In [16]:
#input_shp = f"C:/Geotar/{pilot}/geodata/workspace/roads_clip.shp"
input_shp = f"C:/Geotar/{pilot}/geodata/workspace/health_clip.shp"
# Define NoData value of new raster
NoData_value = -9999

# set the name and location of the output raster file
#dst_filename = f"C:/Geotar/{pilot}/geodata/Processed/250m/dist_roads.tif"
dst_filename = f"C:/Geotar/{pilot}/geodata/Processed/250m/dist_health.tif"

# Open the data source and read in the extent
source_ds = input_shp
pixel_size = 0.0022457882102988 #250m

vector_ds  = ogr.Open(mask_shp)
shp_layer = vector_ds.GetLayer()

xmin, xmax, ymin, ymax = shp_layer.GetExtent()

# check if the output file already exists, and delete it if it does
if os.path.exists(dst_filename):
    drv = gdal.GetDriverByName('GTiff')
    drv.Delete(output_file)

# rasterize the vectori file suing the spatial resolution defined
ds = gdal.Rasterize(output_file, source_ds, xRes=pixel_size, yRes=pixel_size, 
                    burnValues=1,outputBounds=[xmin, ymin, xmax, ymax], 
                    outputType=gdal.GDT_Byte, allTouched=True)
ds = None
source_ds = None

In [18]:
# open an existing raster file
#output_file = f"C:/Geotar/{pilot}/geodata/workspace/rast_road.tif"
output_file = f"C:/Geotar/{pilot}/geodata/workspace/rast_health.tif"
src_ds = gdal.Open(output_file)

# get the first band of the source raster file
srcband = src_ds.GetRasterBand(1)

# set the name and location of the output raster file
#dst_filename = f"C:/Geotar/{pilot}/geodata/Processed/250m/dist_roads.tif"


# check if the output file already exists, and delete it if it does
if os.path.exists(dst_filename):
    drv = gdal.GetDriverByName('GTiff')
    drv.Delete(dst_filename)

# create a new raster file with the same dimensions and data type as the source raster file
# but with only one band of Float32 data type
drv = gdal.GetDriverByName('GTiff')
dst_ds = drv.Create(dst_filename, 
                    src_ds.RasterXSize, 
                    src_ds.RasterYSize, 1, 
                    gdal.GetDataTypeByName('Float32'))

# set the geotransform and projection of the output raster file
dst_ds.SetGeoTransform(src_ds.GetGeoTransform())
dst_ds.SetProjection(src_ds.GetProjectionRef())

#get the first band of the output raster file
dstband = dst_ds.GetRasterBand(1)

# Compute the proximity of the input raster values to the raster value of 1 
# and write the resulting distances to the output raster file
prox = gdal.ComputeProximity(srcband, dstband, ["VALUES=1", "DISTUNITS=GEO"])

# close the input and output raster files and bands to free up memory
srcband = None
dstband = None
src_ds = None
dst_ds = None
prox = None


In [20]:
src_ds

# Scaling to a fixed range 0-1
#proximity_normalized = (proximity_agg - proximity_agg.min()) / (proximity_agg.max() - proximity_agg.min())
